<center>
<a href="http://www.udem.edu.co/"><img src="img/Escudo.png"></a>
<h1>Reconocimiento de Patrones I y II</h1>
<h3>2018-2</h3>
</center>

## De la teoría a la práctica: representación de textos con la matriz TDM y el esquema TF-IDF

En la sesión anterior vimos, tanto desde la teoría como en la práctica, como representar textos vectorialmente, usando patrones morfosintácticos. Vimos también, desde la teoría, la representación de textos usando el esquema de pesado TF-IDF. De igual manera, vimos dos representaciones muy importantes en la actualidad en el reconocimiento de patrones en textos: la matriz de términos en documentos (TDM) y la matriz de términos en contextos (TTM), también conocida como representación de palabras en vectores semánticos o word embeddings.

En la sesión de hoy aprenderemos como llevar a la práctica la representación de textos como vectores usando TF-IDF y la matriz de términos en documentos TDM (para word embedding se recomienda leer acerca de word2vec (Mikolov, T. et. al, 2013) y mirar los tutoriales de la librería <a href="https://radimrehurek.com/gensim/models/word2vec.html">gensim</a> (Khosrovian, K. et. al, 2008)  para llevar esta representación a la práctica).

Para realizar la representación de textos con TDM y TF-IDF, vamos a usar la librería sklearn, especificamente CountVectorizer para TDM y TfidfVectorizer para TF-IDF. La documentación de ambos se puede encontrar en los siguientes enlaces:

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

### Trabajando sobre dos bases de datos: noticias de periódico y twits

In [27]:
# -*- coding: utf-8 -*-

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Cargando los datos desde el archivo DB.txt (Noticias de periódico)
db = open('DB.txt', 'r', encoding="utf-8")
db_data = []
db_target = []
print('Cargando datos desde: DB.txt . . .')
for l in db:
    db_data.append(l.split('\t')[0])
    db_target.append(int(l.split('\t')[1][:-1]))
db.close()
print('Datos cargados.')
#print(db_data[0])
#print(db_target)  #Las clases son 1->textos positivos; 0->tetos negativos


Cargando datos desde: DB.txt . . .
Datos cargados.


In [28]:
# -*- coding: utf-8 -*-
# Cargando la base de datos de Twitter

#Complete el código para cargar la BD.

#### Hay desbalance en la base de datos? Recuerde que en la práctica esto hay que manejarlo.

In [29]:
#print(len(db_data))
#print(len(db_data_twits))

### Se carga el modelo CountVectorizer en la variable vector

In [30]:
vector = CountVectorizer(ngram_range=(1,2))    #Matriz de términos en documentos
#vector = TfidfVectorizer()   #Esquema TF-IDF

### El modelo aprende el vocabulario desde el corpus

In [31]:
vector.fit(db_data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

### Se caracterizan los documentos como vectores a través de TDM

In [32]:
bow = vector.transform(db_data)

In [33]:
bow.shape

(205, 80592)

In [34]:
#print(bow[0,:])     #Analice el formato de la matriz de términos en documentos

### Se cargan las clases (etiqueteas) de las muestras en la variable y

In [35]:
y = db_target

Hasta aquí, ya se tiene lista la representación de los documentos a partir de la matriz de términos en documentos. Para caracterizar los documentos usando el esquema de pesado TF-IDF, simplemente se debe ir a la celda donde se cargó el modelo CountVectorizer y comentar esa línea, y descomentar la línea donde se carga el modelo TfidfVectorizer.

En este punto estamos listos para entrenar y validar diferentes modelos de aprendizaje automático y hacer el reconocimiento de patrones que permita clasificar textos como positivos o negativos, tarea que se conoce como análisis de sentimientos.

## Recordemos la teoría de varios modelos básicos de aprendizaje automático para hacer reconocimiento de patrones.

Desarrollo de la teoría en clase.

## Análisis de sentimientos con diferentes modelos de aprendizaje automático usando Sklearn

### Regresión logística

In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

def error_measures(Yestimado, Yteorico):
    
    CM = confusion_matrix(Yteorico, Yestimado)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    esp = TN/(TN+FP)
    
    return sens, esp

lr=LogisticRegression()
acc = []
sens = []
esp = []

for i in range(100):
    
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(bow,y)   #Realiza una única partición
                                                          #de la base de datos.
    
    lr.fit(Xtrain,Ytrain)
    Yest = lr.predict(Xtest)
    s, e = error_measures(Yest,Ytest)
    sens.append(s); esp.append(e)
    acc.append(lr.score(Xtest,Ytest))

print("Muestras training: ", round((np.size(Xtrain,0)*100)/205), "%")
print("Muestras testing: ", round((np.size(Xtest,0)*100)/205), "%")
    
print("\nResultados con Regresión logística (Lineal)\n")
print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
print("Especificity: ", np.mean(esp), "+/-", np.std(esp))


Muestras training:  75 %
Muestras testing:  25 %

Resultados con Regresión logística (Lineal)

Accuracy:  0.7925 +/- 0.0535220327357
Sensitivity:  0.83857344296 +/- 0.0727854359966
Especificity:  0.743977636036 +/- 0.0798106218797


## Ejercicios

1. Complete las siguientes celdas de código para probar el reconocimiento de patrones (sentiment analysis) con los modelos KNN, árboles de decisión, Random Forest y ANN (multi-layer perceptron).

2. Efectue el reconocimiento de patrones con los mismo modelos pero usando la base de datos de twits.

### K vecinos más cercanos (K-nearest neighbors)

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN

knn=KNN(n_neighbors=5)
acc = []
sens = []
esp = []

for i in range(100):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(bow,y)

    knn.fit(Xtrain, Ytrain)
    Yest = knn.predict(Xtest)
    s, e = error_measures(Yest,Ytest)
    sens.append(s); esp.append(e)
    acc.append(knn.score(Xtest,Ytest))

print("\nResultados con K vecinos más cercanos\n")
print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
print("Especificity: ", np.mean(esp), "+/-", np.std(esp))


Resultados con K vecinos más cercanos

Accuracy:  0.674038461538 +/- 0.0626920127412
Sensitivity:  0.778575294111 +/- 0.0848182121972
Especificity:  0.56102816433 +/- 0.11792876454


### Árboles de decisión

In [43]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier as DT

clf = DT(min_samples_leaf=10)

acc = []
sens = []
esp = []

for i in range(100):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(bow,y)

    clf.fit(Xtrain, Ytrain)
    Yest = knn.predict(Xtest)
    s, e = error_measures(Yest,Ytest)
    sens.append(s); esp.append(e)
    acc.append(clf.score(Xtest,Ytest))

#Complete el código para Árboles de decisión

print("Resultados con Árbol de decisión\n")
print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
print("Especificity: ", np.mean(esp), "+/-", np.std(esp)) 


Resultados con Árbol de decisión

Accuracy:  0.641923076923 +/- 0.0700158477411
Sensitivity:  0.912173001682 +/- 0.0437040849392
Especificity:  0.650556947884 +/- 0.0861469311295


### Random Forest

In [39]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RF

acc = []
sens = []
esp = []

#Complete el código para Árboles de decisión

print("Resultados con Random Forest\n")
print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
print("Especificity: ", np.mean(esp), "+/-", np.std(esp))


Resultados con Random Forest

Accuracy:  nan +/- nan
Sensitivity:  nan +/- nan
Especificity:  nan +/- nan


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Feed-Fordward Neural Networks (Multi-layer perceptron)

In [40]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Perceptron as MLP

acc = []
sens = []
esp = []

#Complete el código para ANN (Multilayer perceptron)

print("Resultados con Perceptróin\n")
print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
print("Especificity: ", np.mean(esp), "+/-", np.std(esp))


Resultados con Perceptróin

Accuracy:  nan +/- nan
Sensitivity:  nan +/- nan
Especificity:  nan +/- nan


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Referencias

Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.

Khosrovian, K., Pfahl, D., & Garousi, V. (2008, May). GENSIM 2.0: a customizable process simulation model for software process evaluation. In International Conference on Software Process (pp. 294-306). Springer, Berlin, Heidelberg.